In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
google_model = Word2Vec.load("../data/model_word2vec.bin")

In [12]:
google_model.wv.most_similar(positive=["SQL"])

KeyError: "Key 'SQL' not present"

In [4]:
file_dataframe = "../data/recommender_resumes_train.pkl"

df = pd.read_pickle(file_dataframe)
df.head(300)

,_id,text,class,raw_text
0,bf8217e0-4fe8-44e1-8446-637ce8f8fb93,international business development manager sum...,estandar\BUSINESS-DEVELOPMENT,INTERNATIONAL BUSINESS DEVELOPMENT MANAGER\nSu...
1,09d9e88a-3b77-4d38-abd9-ce1e4b76d237,hr generalist experience current hr generalist...,estandar\HR,HR GENERALIST\nExperience\n08/2017 to Current\...
2,dff61afa-9949-4d15-9dd9-3251c9836e7c,accountant highlight microsoft office intermed...,estandar\ACCOUNTANT,ACCOUNTANT\nHighlights\nMicrosoft Office : Int...
3,2c9ab807-c1e2-49f0-a4fd-f3780d907470,accountant skill quickbooks sage lacerte intui...,estandar\ACCOUNTANT,"ACCOUNTANT\nSkills\nQuickBooks, Sage, Lacerte,..."
4,782e5b2c-c995-4cc4-a25b-6393b886a4f0,none,estandar\BUSINESS-DEVELOPMENT,None
...,...,...,...,...
295,c3490588-947d-4715-be3e-157925cc0900,mortgage banking executive analyst summary foc...,estandar\BANKING,MORTGAGE BANKING EXECUTIVE ANALYST\nSummary\n\...
296,945b04ec-cbec-4195-88ca-01d5d727a484,mechanical engineer summary year month experie...,estandar\AVIATION,MECHANICAL ENGINEER\nSummary\n\n5 years and 9 ...
297,aef917ca-10df-4881-9ce1-05f60d06f52e,finance business partner summary strategic ana...,estandar\FINANCE,FINANCE BUSINESS PARTNER\nSummary\nStrategic a...
298,976e74ef-4408-4dcd-806a-05d8ecf1163a,it technology specialist professional summary ...,estandar\INFORMATION-TECHNOLOGY,IT TECHNOLOGY SPECIALIST\nProfessional Summary...


In [5]:
# Generate the average word2vec for the each book description

def vectors(x):

    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description
    for line in df['text']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model.wv[word]
                else:
                    avgword2vec = avgword2vec + google_model.wv[word]

        if avgword2vec is not None:
            avgword2vec = avgword2vec / count

            word_embeddings.append(avgword2vec)

In [6]:
# Recommending the Top 5 similar books

def recommendations(title):

    # Calling the function vectors

    vectors(df)

    # finding cosine similarity for the vectors

    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)

    # taking the title and book image link and store in new data frame called books
    books = df[['_id','raw_text']]
    #Reverse mapping of the index
    indices = pd.Series(df.index, index = df['_id'].values.astype('str')).drop_duplicates()

    idx = indices[title]

    #print(len(word_embeddings))
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    #print(sim_scores)
    sim_scores = sim_scores[1:6] # DESDE 1 PARA OCULTAR LA OFERTA
    book_indices = [i[0] for i in sim_scores]
    scores_values = [i[1] for i in sim_scores]
    recommend = books.iloc[book_indices]

    output = []

    countIndex = 0
    for index, row in recommend.iterrows():

        #print(row['title'] )
        #print(scores_values[countIndex])

        output.append([countIndex + 1, row['raw_text'] ,row['_id'] , scores_values[countIndex], index])

        countIndex+=1


        #response = requests.get(row['image_link'])
        #img = Image.open(BytesIO(response.content))
        #plt.figure()
        #plt.imshow(img)
        #plt.title(row['title'])

    result = pd.DataFrame(output, columns=["rank", "common_texts", "title", "cosine_similarity", "index_df"])

    return result

In [7]:
result = recommendations("d59170ea-e3cf-41a3-982b-cf163efa2b4b")
result

KeyError: 'd59170ea-e3cf-41a3-982b-cf163efa2b4b'

In [ ]:
print(df["raw_text"].iloc[251])